In [99]:
from bs4 import BeautifulSoup
import re
from requests import request
import os
import pandas as pd

In [100]:
os.chdir('D:/ml-25m')

In [89]:
movies = pd.read_csv('movies.csv')

In [4]:
movies.loc[movies.genres.str.contains('no genres listed'), 'genres'] = np.nan

In [5]:
links = pd.read_csv('links.csv')

In [6]:
imdb = pd.read_csv('title_basics.tsv', na_values='\\N', sep='\t')

C:\Users\USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
imdb.tconst = imdb.tconst.str.replace('tt', '')
imdb.tconst = imdb.tconst.astype(np.int64)

In [8]:
imdb.rename(columns={'tconst':'imdbId'}, inplace=True)

In [9]:
merged = links.merge(imdb, on='imdbId', how='inner')

,movieId,imdbId,tmdbId,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
62319,209157,6671244,499546.0,movie,Wij,Wij,0.0,2018.0,NaN,100.0,Drama
62320,209159,297986,63407.0,movie,Window of the Soul,Janela da Alma,0.0,2001.0,NaN,73.0,Documentary
62321,209163,6755366,553036.0,movie,Bad Poems,Rossz versek,0.0,2018.0,NaN,97.0,"Comedy,Drama"
62322,209169,249603,162892.0,tvMovie,A Girl Thing,A Girl Thing,0.0,2001.0,NaN,237.0,"Comedy,Drama,Romance"
62323,209171,55323,79513.0,movie,Women of Devil's Island,Le prigioniere dell'isola del diavolo,0.0,1962.0,NaN,86.0,"Action,Adventure,Drama"


In [10]:
g_imdb = merged.loc[merged.genres.isnull()==False, ['movieId', 'genres']]
g_imdb.genres = g_imdb.genres.str.replace(',', '|')

In [16]:
movies_merge = movies.merge(g_imdb, on="movieId", how="outer")

In [24]:
movies_merge.loc[(movies_merge.genres_x.isnull()==True) & (movies_merge.genres_y.isnull()==False), 'genres_x'] = movies_merge.loc[(movies_merge.genres_x.isnull()==True) & (movies_merge.genres_y.isnull()==False), 'genres_y']

In [27]:
movies_merge.drop('genres_y', axis=1, inplace=True)
movies_merge.rename(columns={'genres_x':'genres'}, inplace=True)

In [30]:
to_search = list(movies_merge.loc[movies_merge.genres.isnull()==True, 'title'])

In [31]:
def get_link(url):
    resp = request(url=url, headers=headers, method='GET', allow_redirects=False)
    dom = BeautifulSoup(resp.text, 'lxml')
    link = base_url + re.findall(r'<a href="([\D\d\s]+)">', str(dom.select('td > a')[1]))[0]
    return link

In [82]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'
}
base_url =' https://www.imdb.com/'
genres = {}
not_exist = []
for title in to_search:
    try:
        url = base_url + f'find?q={title}&s=tt&ttype=ft&exact=true&ref_=fn_tt_ex'
        link = get_link(url)
    except Exception:
        try:
            url = base_url + f'find?q={title}&s=tt&ttype=ft&ref_=fn_ft'
            link = get_link(url)
        except Exception:
            url = 'https://www.allmovie.com/search/movies/' + title
            resp = request(url=url, headers=headers, method='GET', allow_redirects=False)
            dom = BeautifulSoup(resp.text, 'lxml')
            try: 
                genre = re.findall(r'<a href="[\d\D\s]+">([\d\D\s]+)</a>', str(dom.find_all('div', {'class': 'genres'})[0]))[0].replace(' &amp; ', '|')
            except Exception:
                not_exist.append(title)
                continue
                
    resp = request(url=link, headers=headers, method='GET')
    dom = BeautifulSoup(resp.text, 'lxml')    
    try:
        genre = re.findall(r'\\"genre\\":\[\\"([\d\D\s]+)\\"\]', str(dom))[0].replace('\\",\\"', '|')
    except Exception:
        url = 'https://www.allmovie.com/search/movies/' + title
        resp = request(url=url, headers=headers, method='GET', allow_redirects=False)
        dom = BeautifulSoup(resp.text, 'lxml')
        try: 
            genre = re.findall(r'<a href="[\d\D\s]+">([\d\D\s]+)</a>', str(dom.find_all('div', {'class': 'genres'})[0]))[0].replace(' &amp; ', '|')
        except Exception:
            not_exist.append(title)
            continue
    genres[title]=genre

In [83]:
for k, v in genres.items():
    movies_merge.loc[movies_merge.title ==k, 'genres'] = v

In [87]:
movies = movies_merge[movies_merge.title.isin(not_exist) == False]

In [141]:
movies.genres = movies.genres.str.replace("Music", "Musical")
movies.genres = movies.genres.str.replace("Musicalal", "Musical")

In [142]:
genre_list = ["Action", "Adventure", "Animation", "Biography", "Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
              "Film-Noir", "History", "Horror", "Musical", "Mystery", "Romance", "Sport", "Sci-Fi", "Thriller", "War", "Western"]

In [143]:
genres = movies.genres
to_row = [g.split('|') for g in genres]

In [144]:
movies.genres = to_row

In [146]:
to_drop = []
for i, lst in enumerate(list(movies.genres)):
    for g in lst:
        if g not in genre_list:
            lst.remove(g)
            
    if len(lst) == 0:
        to_drop.append(i)

In [149]:
movies.drop(to_drop, axis=0, inplace=True)

In [96]:
movies.to_csv('ml-25m.csv', index=False)